In [32]:
from json import load
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()

data_path = os.getenv('DATA_PATH')
ambient_dir = f'{data_path}/ambient' 
attack_dir = f'{data_path}/attacks'

df_train = pd.read_parquet(f'{data_path}/ambient/ambient_train_chunks.parquet')
df_test = pd.read_parquet(f'{data_path}/attacks/attacks_test_chunks.parquet')

In [10]:
# Shuffle the DataFrame
df_train = df_train.sample(frac=1).reset_index(drop=True)

# Split into features and labels
X = df_train.drop(columns=['label'])
y = df_train['label']

# Convert to NumPy arrays
X = X.values
y = y.values


N = 11

In [33]:
# Now for test
df_test = df_test.sample(frac=1).reset_index(drop=True)

# Split into features and labels
X_test = df_test.drop(columns=['label'])
y_test = df_test['label']

# Convert to NumPy arrays
X_test = X_test.values
y_test = y_test.values

In [46]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")

# Define custom dataset
class TimeDifferenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create dataset and dataloader
dataset = TimeDifferenceDataset(X, y)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# Create test dataset and dataloader
test_dataset = TimeDifferenceDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

# Define model
class TimeDifferenceModel(nn.Module):
    def __init__(self, input_dim):
        super(TimeDifferenceModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        return self.fc4(x)

model = TimeDifferenceModel(N)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [47]:
from tqdm import tqdm
num_epochs = 10
for epoch in range(num_epochs):
    for batch_X, batch_y in tqdm(dataloader):
        # Forward pass
        outputs = model(batch_X) 
        loss = criterion(outputs.squeeze(), batch_y)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


100%|██████████| 176625/176625 [01:48<00:00, 1628.92it/s]


Epoch [1/10], Loss: 0.0034


100%|██████████| 176625/176625 [01:47<00:00, 1638.70it/s]


Epoch [2/10], Loss: 0.0008


100%|██████████| 176625/176625 [01:51<00:00, 1589.70it/s]


...

Epoch [8/10], Loss: 0.0011


100%|██████████| 176625/176625 [01:48<00:00, 1634.82it/s]


Epoch [9/10], Loss: 0.0012


100%|██████████| 176625/176625 [01:49<00:00, 1616.04it/s]

Epoch [10/10], Loss: 0.0012


In [35]:
torch.save(model, 'model.pth')

In [36]:
for batch_X, batch_y in tqdm(dataloader):
    predictions = model(batch_X)

100%|██████████| 176625/176625 [01:03<00:00, 2779.31it/s]


In [37]:
predictions

tensor([[0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.2636],
        [0.1219],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.0270],
        [0.4655],
        [0.0270],
        [0.0270],
        [0.0270],
        [0